In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
import os
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

data_dir = "./animals10"

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
model = models.alexnet(pretrained=False)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
model.load_state_dict(torch.load("masked_net_trained.pth"))
model.to(device)

for param in model.parameters():
    param.requires_grad = True

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_41968\2260555728.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbit

In [5]:
def evaluate(model, dataloader):
    model.eval()
    correct, total, total_loss = 0, 0, 0.0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item()
            _, pred = out.max(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total, total_loss / len(dataloader)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4) 

epochs = 20
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_acc, _ = evaluate(model, train_loader)
    test_acc, test_loss = evaluate(model, test_loader)

    print(f"\n📊 [Epoch {epoch}]")
    print(f"   🔧 Train Accuracy : {train_acc*100:.2f}% | Train Loss: {train_loss:.4f}")
    print(f"   🧪 Test Accuracy  : {test_acc*100:.2f}% | Test Loss: {test_loss:.4f}")


📊 [Epoch 1]
   🔧 Train Accuracy : 27.63% | Train Loss: 8.7366
   🧪 Test Accuracy  : 26.73% | Test Loss: 2.0784

📊 [Epoch 2]
   🔧 Train Accuracy : 33.29% | Train Loss: 2.0484
   🧪 Test Accuracy  : 31.44% | Test Loss: 1.9593

📊 [Epoch 3]
   🔧 Train Accuracy : 38.45% | Train Loss: 1.9315
   🧪 Test Accuracy  : 36.58% | Test Loss: 1.8479

📊 [Epoch 4]
   🔧 Train Accuracy : 42.03% | Train Loss: 1.8167
   🧪 Test Accuracy  : 39.17% | Test Loss: 1.7668

📊 [Epoch 5]
   🔧 Train Accuracy : 46.58% | Train Loss: 1.7111
   🧪 Test Accuracy  : 42.22% | Test Loss: 1.6650

📊 [Epoch 6]
   🔧 Train Accuracy : 50.28% | Train Loss: 1.6234
   🧪 Test Accuracy  : 44.29% | Test Loss: 1.6038

📊 [Epoch 7]
   🔧 Train Accuracy : 53.81% | Train Loss: 1.5367
   🧪 Test Accuracy  : 47.28% | Test Loss: 1.5399

📊 [Epoch 8]
   🔧 Train Accuracy : 56.42% | Train Loss: 1.4613
   🧪 Test Accuracy  : 48.43% | Test Loss: 1.4961

📊 [Epoch 9]
   🔧 Train Accuracy : 58.98% | Train Loss: 1.3798
   🧪 Test Accuracy  : 51.14% | Test Loss:

In [6]:
#imprimir numero de parametros que están a cero
def count_zero_params(model):
    zero_count = 0
    total_count = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            zero_count += (param.data == 0).sum().item()
            total_count += param.numel()
    return zero_count, total_count

zero_count, total_count = count_zero_params(model)
print(f"\n🔍 Zero Parameters: {zero_count} out of {total_count} ({(zero_count / total_count) * 100:.2f}%)")


🔍 Zero Parameters: 41929626 out of 57035456 (73.52%)
